In [1]:
import collections
import math
import os
import random
from tempfile import gettempdir
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

In [6]:
#1  Donwload dataset
url = 'http://mattmahoney.net/dc/'

# pylint: disable=redefined-outer-name
def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    local_filename = os.path.join(gettempdir(), filename)
    if not os.path.exists(local_filename):
        local_filename, _ = urllib.request.urlretrieve(url + filename,
                                                   local_filename)
    statinfo = os.stat(local_filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + local_filename +
                    '. Can you get to it with a browser?')
    return local_filename


filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [17]:
# read dato to list of strings:
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data

vocabulary = read_data(filename)
print("Data size", len(vocabulary))

Data size 17005207


In [18]:
# 2 build dictionary and replace rare works with UNK token
vocabulary_size = 50000

def build_dataset(words, n_words):
    count = [["UNK", -1]]
    count.extend(collections.Counter(words).most_common(n_words -1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(vocabulary, vocabulary_size)

del vocabulary
print("most common words (+UNK)", count[:5])
print("sample data", data[:10], [reverse_dictionary[i] for i in data[:10]])

data_index = 0

most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
sample data [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


In [19]:
# step 3, generate training batch for skip-gram model:
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
            if data_index == len(data):
                buffer[:] = data[:span]
                data_index = span
            else:
                buffer.append(data[data_index])
                data_index += 1
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

3081 originated -> 12 as
12 as -> 3081 originated
6 a -> 12 as
195 term -> 2 of
2 of -> 3134 abuse
3134 abuse -> 2 of
46 first -> 59 used
59 used -> 46 first


In [31]:
# 4 build and train skip-gram model:
batch_size = 128
embedding_size = 128 # dim embedding vec
skip_window = 1 #words left and right
num_skips = 2 # times to reuse input to generate label
num_sampled = 64 # num negative examples to sample

valid_size = 16
valid_window = 100
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

graph = tf.Graph()

with graph.as_default():
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])    
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    with tf.device("/cpu:0"):
        embeddings = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)
        
        # NCE loss:
        nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size], stddev=0.1/math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
        
        # compute loss:
        loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights, biases=nce_biases, labels=train_labels,
                                           inputs=embed, num_sampled=num_sampled, num_classes=vocabulary_size))
        
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
        
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
        similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
        
        init = tf.global_variables_initializer()

In [39]:
# begin training
num_steps = 100001
with tf.Session(graph=graph) as session:
    init.run()
    print("initialized")
    
    average_loss = 0
    for step in range(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}
        
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val
        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0
        
    # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

initialized
Average loss at step  0 :  319.260498047
Nearest to be: schrader, gnu, digitized, sultan, girls, flogging, haer, durand,
Nearest to d: pantomime, katharina, kilby, freebsd, fractured, nzwani, salic, infrastructures,
Nearest to its: secretion, ithaca, moustache, infinitesimal, korca, chording, voiced, zil,
Nearest to world: discontinuities, adaptive, crts, pavilion, quantify, www, permissive, narcotics,
Nearest to often: vci, apprehension, porto, hoosier, bays, glutamic, plague, brugge,
Nearest to it: tonnage, redskins, monoids, dormitory, ellipsis, olomouc, dks, dmd,
Nearest to no: wads, dysfunction, kapital, stanshall, afrika, provincial, erv, dachshund,
Nearest to other: grab, chainsaw, murdock, ve, scams, ez, ascertaining, gil,
Nearest to during: suva, bridgeport, parachutes, permanent, simulcast, reversible, russel, purdue,
Nearest to and: outlines, coercive, enhances, finlay, tomography, updating, beauchamp, lodges,
Nearest to or: conflation, excommunication, neighborh

Average loss at step  62000 :  6.52681374359
Average loss at step  64000 :  6.61412830722
Average loss at step  66000 :  6.42271772087
Average loss at step  68000 :  6.54447826946
Average loss at step  70000 :  6.56572987747
Nearest to be: have, by, was, were, are, is, been, being,
Nearest to d: b, e, and, UNK, six, sh, zero, five,
Nearest to its: their, his, the, some, her, these, a, our,
Nearest to world: us, one, project, century, UNK, god, japanese, series,
Nearest to often: also, it, which, that, even, not, sometimes, generally,
Nearest to it: he, this, which, there, they, not, also, that,
Nearest to no: this, also, a, which, it, or, now, nine,
Nearest to other: some, many, two, most, these, are, three, non,
Nearest to during: in, from, with, on, of, by, was, after,
Nearest to and: or, in, seven, six, eight, of, nine, but,
Nearest to or: and, eight, six, seven, four, nine, five, zero,
Nearest to about: on, three, four, five, within, company, nine, two,
Nearest to states: english, 